In [85]:
import numpy as np
import pandas as pd
import random
import time
import math
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import init
from dateutil import parser
from pathlib import Path
import json 
import shutil
import logging
logger = logging.getLogger(str(os.getpid()))

from wattile.data_reading import read_dataset_from_file
from wattile.buildings_processing import correct_predictor_columns, correct_timestamps, resample_or_rolling_stats, timelag_predictors, timelag_predictors_target, roll_predictors_target
from wattile.time_processing import add_processed_time_columns
PROJECT_DIRECTORY = Path().resolve().parent.parent

# reading configs

In [86]:
"""
For this example, we will be using the default configs.
Check out the docs for an explaination of each config.
"""
##################################################################################
# choose the configs file to use as an input
##################################################################################
# main configs file
with open(PROJECT_DIRECTORY / "wattile" / "configs" / "configs.json", "r") as f:
    configs = json.load(f)
##################################################################################
# code testing configs file
# with open(PROJECT_DIRECTORY / "tests" / "fixtures" / "test_configs.json", "r") as f:
#     configs = json.load(f)
##################################################################################

exp_dir = PROJECT_DIRECTORY / "notebooks" / "exp_dir"
if exp_dir.exists():
    shutil.rmtree(exp_dir)
exp_dir.mkdir()

configs["data_input"]["exp_dir"] = str(PROJECT_DIRECTORY / exp_dir)
configs["data_input"]["data_dir"] = str(PROJECT_DIRECTORY / "data" / "Synthetic Site")
# configs["data_input"]["data_dir"] = str(PROJECT_DIRECTORY / ".." / "intelligentcampus-feature-eng" / "data" / "Cafe")
# configs["data_input"]["data_config"] = "Cafe Config.json"
# configs["data_input"]["predictor_columns"] = [
#     "SRRL BMS Atmospheric Electric Field",
#     "SRRL BMS Barometric Pressure",
#     "SRRL BMS Dew Point Temperature",
#     "SRRL BMS Diffuse Horizontal Irradiance",
#     "SRRL BMS Direct Normal Irradiance",
#     "SRRL BMS Dry Bulb Temperature",
#     "SRRL BMS DWIR",
# #     "SRRL BMS Global 40Â° South Irradiance",
# #     "SRRL BMS Global 90Â° South Irradiance",
# #     "SRRL BMS Global Horizontal Irradiance",
# #     "SRRL BMS Global Illuminance",
#     "SRRL BMS Global Normal Irradiance",
# #     "SRRL BMS Global UV Index",
#     "SRRL BMS Opaque Cloud Cover",
# #     "SRRL BMS Peak Wind Speed at 19'",
# #     "SRRL BMS Peak Wind Speed at 6'",
# #     "SRRL BMS Rainfall",
#     "SRRL BMS Relative Humidity",
# #     "SRRL BMS Snow Depth",
#     "SRRL BMS Total Cloud Cover",
#     "SRRL BMS UWIR",
#     "SRRL BMS UWSW",
#     "SRRL BMS Wet Bulb Temperature",
# #     "SRRL BMS Wind Direction at 19'",
# #     "SRRL BMS Wind Direction at 6'",
# #     "SRRL BMS Wind Speed at 19'",
# #     "SRRL BMS Wind Speed at 6'",
# ]
# configs["data_input"]["target_var"] = "Cafe Whole Building Real Power Total"

configs

{'data_input': {'data_dir': 'C:\\Users\\JKIM4\\Documents\\GitHub\\intelligentcampus-pred-analytics\\data\\Synthetic Site',
  'data_config': 'Synthetic Site Config.json',
  'start_time': '2018-01-01T00:00:00-07:00',
  'end_time': '2022-01-01T00:00:00-07:00',
  'predictor_columns': ['Synthetic Weather Station Dew Point Temperature',
   'Synthetic Weather Station Diffuse Horizontal Irradiance',
   'Synthetic Weather Station Direct Normal Irradiance',
   'Synthetic Weather Station Dry Bulb Temperature',
   'Synthetic Weather Station Global Horizontal Irradiance',
   'Synthetic Weather Station Relative Humidity',
   'Synthetic Weather Station Wind Speed'],
  'target_var': 'Synthetic Site Electricity Main Total Power',
  'exp_dir': 'C:\\Users\\JKIM4\\Documents\\GitHub\\intelligentcampus-pred-analytics\\notebooks\\exp_dir'},
 'data_output': {'exp_dir': 'exp_dir',
  'plot_comparison': True,
  'plot_comparison_portion_start': 0.0,
  'plot_comparison_portion_end': 1.0},
 'data_processing': {'fea

# reading data

In [71]:
configs["target_feat_name"] = [configs["data_input"]["target_var"]]
data = read_dataset_from_file(configs)
data

,Synthetic Weather Station Dew Point Temperature,Synthetic Weather Station Diffuse Horizontal Irradiance,Synthetic Weather Station Direct Normal Irradiance,Synthetic Weather Station Dry Bulb Temperature,Synthetic Weather Station Global Horizontal Irradiance,Synthetic Weather Station Relative Humidity,Synthetic Weather Station Wind Speed,Synthetic Site Electricity Main Total Power
Timestamp,,,,,,,,
2021-12-01 07:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,104.800003
2021-12-01 07:01:00+00:00,15.183906,-1.496226,0.207964,56.174000,-1.841866,17.879999,11.265539,96.650002
2021-12-01 07:02:00+00:00,14.891221,-1.441191,0.457520,55.886002,-1.873579,17.809999,12.777752,96.199997
2021-12-01 07:03:00+00:00,14.836164,-1.371754,0.473117,55.796001,-1.860084,17.820000,12.694983,95.599998
2021-12-01 07:04:00+00:00,14.782966,-1.317349,0.545903,55.723999,-1.843271,17.820000,11.632407,100.650002
...,...,...,...,...,...,...,...,...
2021-12-08 06:55:00+00:00,11.210565,-1.487212,-0.254757,42.285198,-1.814939,24.680000,0.000000,109.664803
2021-12-08 06:56:00+00:00,11.190062,-1.445182,-0.233960,41.997200,-1.852901,24.930000,0.000000,107.002800
2021-12-08 06:57:00+00:00,11.223961,-1.396302,-0.145575,41.669601,-1.841248,25.290001,3.545647,106.480400


# plot setting

In [3]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.colors import n_colors
from plotly.validators.scatter.marker import SymbolValidator
import plotly.io as pio
os.environ['path'] += r';C:/Users/JKIM4/Downloads/vips-dev-w64-all-8.11.0/vips-dev-8.11/bin'
import pyvips
import copy
import random

random.seed(1)
path = "../../tests/fixtures"
colorscale = "Earth"

# data processing 1

In [72]:
# Add time-based features
data = add_processed_time_columns(data, configs)

data

,Synthetic Weather Station Dew Point Temperature,Synthetic Weather Station Diffuse Horizontal Irradiance,Synthetic Weather Station Direct Normal Irradiance,Synthetic Weather Station Dry Bulb Temperature,Synthetic Weather Station Global Horizontal Irradiance,Synthetic Weather Station Relative Humidity,Synthetic Weather Station Wind Speed,Synthetic Site Electricity Main Total Power,sin_HOD,cos_HOD,...,DOW_binary_reg_6,DOW_binary_fuzzy_0,DOW_binary_fuzzy_1,DOW_binary_fuzzy_2,DOW_binary_fuzzy_3,DOW_binary_fuzzy_4,DOW_binary_fuzzy_5,DOW_binary_fuzzy_6,sin_MOY,cos_MOY
Timestamp,,,,,,,,,,,,,,,,,,,,,
2021-12-01 07:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,104.800003,0.965926,-0.258819,...,0,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589
2021-12-01 07:01:00+00:00,15.183906,-1.496226,0.207964,56.174000,-1.841866,17.879999,11.265539,96.650002,0.964787,-0.263031,...,0,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589
2021-12-01 07:02:00+00:00,14.891221,-1.441191,0.457520,55.886002,-1.873579,17.809999,12.777752,96.199997,0.963630,-0.267238,...,0,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589
2021-12-01 07:03:00+00:00,14.836164,-1.371754,0.473117,55.796001,-1.860084,17.820000,12.694983,95.599998,0.962455,-0.271440,...,0,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589
2021-12-01 07:04:00+00:00,14.782966,-1.317349,0.545903,55.723999,-1.843271,17.820000,11.632407,100.650002,0.961262,-0.275637,...,0,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-08 06:55:00+00:00,11.210565,-1.487212,-0.254757,42.285198,-1.814939,24.680000,0.000000,109.664803,0.971342,-0.237686,...,0,0.0,0.0,0.750000,0.250000,0.0,0.0,0.0,-0.385663,0.922640
2021-12-08 06:56:00+00:00,11.190062,-1.445182,-0.233960,41.997200,-1.852901,24.930000,0.000000,107.002800,0.970296,-0.241922,...,0,0.0,0.0,0.750000,0.250000,0.0,0.0,0.0,-0.385663,0.922640
2021-12-08 06:57:00+00:00,11.223961,-1.396302,-0.145575,41.669601,-1.841248,25.290001,3.545647,106.480400,0.969231,-0.246153,...,0,0.0,0.0,0.750000,0.250000,0.0,0.0,0.0,-0.385663,0.922640


### feat_time paramters

In [ ]:
list_line = ["solid", "dot", "dash", "longdash", "dashdot", "longdashdot"]

n_colors = len(list_line)
list_colors = px.colors.sample_colorscale(colorscale, [n/(n_colors -1) for n in range(n_colors)])
random.shuffle(copy.deepcopy(list_colors))

df_feat_time = data.loc[:, data.columns.str.contains("MOY|DOW|HOD")]
df_feat_time

dict_daterange = {
    "HOD":["2019-12-02 00:00:00+00:00","2019-12-03 00:00:00+00:00"],
    "DOW":["2019-12-02 00:00:00+00:00","2019-12-10 00:00:00+00:00"],
    "MOY":["2019-01-01 00:00:00+00:00","2020-01-01 00:00:00+00:00"]
}

dict_feat_include = {
    "HOD":[
        "sin_HOD",
        "cos_HOD",
        "HOD_binary_reg_10",
        "HOD_binary_reg_20",
        "HOD_binary_fuzzy_13",
        "HOD_binary_fuzzy_23"
    ],
    "DOW":[
        "DOW_binary_reg_0",
        "DOW_binary_reg_2",
        "DOW_binary_reg_4",
        "DOW_binary_fuzzy_1",
        "DOW_binary_fuzzy_3",
        "DOW_binary_fuzzy_5",
    ],
    "MOY":[
        "sin_MOY",
        "cos_MOY",
    ],
}

i=0
for feat in dict_daterange.keys():
    fig = go.Figure()
    
    df_temp = data.loc[dict_daterange[feat][0]:dict_daterange[feat][1], data.columns.str.contains(feat)][dict_feat_include[feat]]
    
    if feat == "DOW":
        df_temp = df_temp.resample("60min").mean()
    
    i_label = 0
    for col in df_temp.columns:
        
        fig.add_trace(go.Scatter(
            mode="lines",
            x=df_temp.index.values,
            y=df_temp[col].values,
            name=col,
            line=dict(
                color=list_colors[i_label],
                dash=list_line[i_label],
            ),
        ))
        
        i_label+=1

    fig.update_layout(
        width=800,
        height=250,
        margin=dict(
            l=0,
            r=0,
            t=30,
            b=0,
        ),
        plot_bgcolor="rgb(245,245,245)",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1,
            xanchor="center",
            x=0.5,
        )
    )

    filename = "example_feat_time_{}".format(feat)
    pio.write_image(fig, path + "/" + filename + ".svg")
    # https://stackoverflow.com/questions/51450134/how-to-convert-svg-to-png-or-jpeg-in-python
    image = pyvips.Image.thumbnail(path + "/" + filename + ".svg", 3000)
    image.write_to_file(path + "/" + filename + ".png")

    fig.show()
    
#     if i== 1:
#         break
        
    i+=1

### resample paramters

In [60]:
configs["data_input"]["target_var"] = "Synthetic Site Electricity Main Total Power"

configs["data_processing"]["resample"]["bin_interval"] = "3min"
configs["data_processing"]["resample"]["bin_closed"] = "right"
configs["data_processing"]["resample"]["bin_label"] = "right"

configs["data_processing"]["feat_stats"]["active"] = False

In [61]:
df_test = pd.read_csv("../../tests/fixtures/rolling_stats_input_w_target.csv", index_col=0)
df_test.index = pd.to_datetime(df_test.index)
df_test = df_test.loc[:, df_test.columns!="var2"]
df_test

,var1,Synthetic Site Electricity Main Total Power
ts,,
2022-07-12 07:00:00+06:00,NaN,1.0
2022-07-12 07:01:53+06:00,1.5,1.2
2022-07-12 07:03:17+06:00,2.2,1.3
2022-07-12 07:04:02+06:00,0.9,NaN
2022-07-12 07:04:59+06:00,3.6,2.5
2022-07-12 07:05:00+06:00,NaN,2.3
2022-07-12 07:06:22+06:00,3.3,4.3
2022-07-12 07:09:46+06:00,2.3,3.4
2022-07-12 07:10:00+06:00,NaN,NaN


In [62]:
df_resample = resample_or_rolling_stats(df_test, configs)
df_resample = df_resample.add_suffix("_resampled | {} interval | {} closed | {} label".format(
    configs["data_processing"]["resample"]["bin_interval"],
    configs["data_processing"]["resample"]["bin_closed"],
    configs["data_processing"]["resample"]["bin_label"]
))
df_resample

,var1_resampled | 3min interval | right closed | right label,Synthetic Site Electricity Main Total Power_resampled | 3min interval | right closed | right label
ts,,
2022-07-12 07:00:00+06:00,NaN,1.0
2022-07-12 07:03:00+06:00,1.5,1.2
2022-07-12 07:06:00+06:00,3.6,2.3
2022-07-12 07:09:00+06:00,3.3,4.3
2022-07-12 07:12:00+06:00,1.3,3.4
2022-07-12 07:15:00+06:00,4.1,1.1


In [63]:
df_test = df_test.loc[:, df_test.columns=="var1"]
df_resample = df_resample.loc[:, df_resample.columns.str.contains("var1")]

In [64]:
list_symbol = ["circle-open-dot", "square-open-dot"]

n_colors = len(list_symbol)
list_colors = px.colors.sample_colorscale(colorscale, [n/(n_colors -1) for n in range(n_colors)])
random.shuffle(copy.deepcopy(list_colors))

fig = go.Figure()
    
i_label = 0
for col in df_test.columns:

    fig.add_trace(go.Scatter(
        mode="markers",
        x=df_test.index.values,
        y=df_test[col].values,
        name=col,
        marker=dict(
            symbol=list_symbol[i_label],
            size=15,
            line_width=2,
        ),
        line=dict(
            color=list_colors[i_label],
        ),
    ))

    i_label+=1
    
for col in df_resample.columns:

    fig.add_trace(go.Scatter(
        mode="markers",
        x=df_resample.index.values,
        y=df_resample[col].values,
        name=col,
        marker=dict(
            symbol=list_symbol[i_label],
            size=10,
            line_width=2,
        ),
        line=dict(
            color=list_colors[i_label],
        ),
    ))

    i_label+=1

fig.update_layout(
    width=800,
    height=250,
    margin=dict(
        l=0,
        r=0,
        t=30,
        b=0,
    ),
    plot_bgcolor="rgb(245,245,245)",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="center",
        x=0.5,
    )
)

fig.update_xaxes(
    range=["2022-07-12 00:59:00+06:00","2022-07-12 01:16:00+06:00"],
    dtick=1000*60,
    showgrid=True,
    gridwidth=2, 
)

filename = "example_resample_{}-closed_{}-label".format(
    configs["data_processing"]["resample"]["bin_closed"],
    configs["data_processing"]["resample"]["bin_label"]
)
pio.write_image(fig, path + "/" + filename + ".svg")
# https://stackoverflow.com/questions/51450134/how-to-convert-svg-to-png-or-jpeg-in-python
image = pyvips.Image.thumbnail(path + "/" + filename + ".svg", 3000)
image.write_to_file(path + "/" + filename + ".png")

fig.show()

# #     if i== 1:
# #         break

# i+=1

### feat_stats

In [103]:
configs["data_input"]["target_var"] = "Synthetic Site Electricity Main Total Power"

configs["data_processing"]["resample"]["bin_interval"] = "3min"
configs["data_processing"]["resample"]["bin_closed"] = "right"
configs["data_processing"]["resample"]["bin_label"] = "right"

configs["data_processing"]["feat_stats"]["active"] = True
configs["data_processing"]["feat_stats"]["window_width"] = "3min"

In [104]:
df_test = pd.read_csv("../../tests/fixtures/rolling_stats_input_w_target.csv", index_col=0)
df_test.index = pd.to_datetime(df_test.index)
df_test = df_test.loc[:, df_test.columns!="var2"]
df_test

,var1,Synthetic Site Electricity Main Total Power
ts,,
2022-07-12 07:00:00+06:00,NaN,1.0
2022-07-12 07:01:53+06:00,1.5,1.2
2022-07-12 07:03:17+06:00,2.2,1.3
2022-07-12 07:04:02+06:00,0.9,NaN
2022-07-12 07:04:59+06:00,3.6,2.5
2022-07-12 07:05:00+06:00,NaN,2.3
2022-07-12 07:06:22+06:00,3.3,4.3
2022-07-12 07:09:46+06:00,2.3,3.4
2022-07-12 07:10:00+06:00,NaN,NaN


In [105]:
df_resample = resample_or_rolling_stats(df_test, configs)
df_resample = df_resample.add_suffix(" | {} window".format(
    configs["data_processing"]["feat_stats"]["window_width"]
))
df_resample

,var1_min | 3min window,var1_max | 3min window,var1_mean | 3min window,Synthetic Site Electricity Main Total Power | 3min window
ts,,,,
2022-07-12 07:00:00+06:00,NaN,NaN,NaN,1.0
2022-07-12 07:03:00+06:00,1.5,1.5,1.500000,1.2
2022-07-12 07:06:00+06:00,0.9,3.6,2.233333,2.3
2022-07-12 07:09:00+06:00,3.3,3.3,3.300000,4.3
2022-07-12 07:12:00+06:00,1.3,2.3,1.800000,3.4
2022-07-12 07:15:00+06:00,4.1,4.3,4.200000,1.1


In [106]:
df_test = df_test.loc[:, df_test.columns=="var1"]
df_resample = df_resample.loc[:, df_resample.columns.str.contains("var1")]

In [107]:
list_symbol = ["circle-open-dot", "square-open-dot", "diamond-open-dot", "x-open-dot"]

n_colors = len(list_symbol)
list_colors = px.colors.sample_colorscale(colorscale, [n/(n_colors -1) for n in range(n_colors)])
random.shuffle(copy.deepcopy(list_colors))

fig = go.Figure()
    
i_label = 0
for col in df_test.columns:

    fig.add_trace(go.Scatter(
        mode="markers",
        x=df_test.index.values,
        y=df_test[col].values,
        name=col,
        marker=dict(
            symbol=list_symbol[i_label],
            size=15,
            line_width=2,
        ),
        line=dict(
            color=list_colors[i_label],
        ),
    ))

    i_label+=1
    
for col in df_resample.columns:

    fig.add_trace(go.Scatter(
        mode="markers",
        x=df_resample.index.values,
        y=df_resample[col].values,
        name=col,
        marker=dict(
            symbol=list_symbol[i_label],
            size=10,
            line_width=2,
        ),
        line=dict(
            color=list_colors[i_label],
        ),
    ))

    i_label+=1

fig.update_layout(
    width=800,
    height=250,
    margin=dict(
        l=0,
        r=0,
        t=30,
        b=0,
    ),
    plot_bgcolor="rgb(245,245,245)",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="center",
        x=0.5,
    )
)

fig.update_xaxes(
    range=["2022-07-12 00:59:00+06:00","2022-07-12 01:16:00+06:00"],
    dtick=1000*60,
    showgrid=True,
    gridwidth=2, 
)

filename = "example_feat_stats"
pio.write_image(fig, path + "/" + filename + ".svg")
# https://stackoverflow.com/questions/51450134/how-to-convert-svg-to-png-or-jpeg-in-python
image = pyvips.Image.thumbnail(path + "/" + filename + ".svg", 3000)
image.write_to_file(path + "/" + filename + ".png")

fig.show()

# #     if i== 1:
# #         break

# i+=1

### feat_timelag

In [108]:
configs["data_processing"]["feat_timelag"]["lag_interval"] = "60min"
configs["data_processing"]["feat_timelag"]["lag_count"] = 5

In [109]:
timestamp_start = "2021-12-02 00:00:00+00:00"
timestamp_end = "2021-12-03 00:00:00+00:00"

var = "Synthetic Weather Station Dry Bulb Temperature"

data_temp = data.loc[timestamp_start:timestamp_end, (data.columns==var)|(data.columns==configs["data_input"]["target_var"])]
data_temp.columns = ['var1', configs["data_input"]["target_var"]]

In [110]:
data_feat_stats = timelag_predictors(data_temp, configs)
data_feat_stats = data_feat_stats.loc[:, data_feat_stats.columns.str.contains("var1")]
data_feat_stats = data_feat_stats.iloc[:, ::-1]
data_feat_stats

,var1,var1_lag1,var1_lag2,var1_lag3,var1_lag4,var1_lag5
Timestamp,,,,,,
2021-12-02 05:00:00+00:00,61.358002,62.132000,61.574001,61.807999,61.844002,62.978001
2021-12-02 05:01:00+00:00,61.375999,62.113998,61.610001,61.844002,61.844002,62.978001
2021-12-02 05:02:00+00:00,61.430000,62.042000,61.664001,61.826000,61.862000,62.942001
2021-12-02 05:03:00+00:00,61.430000,61.987999,61.700001,61.844002,61.790001,62.905998
2021-12-02 05:04:00+00:00,61.214001,62.186001,61.754002,61.880001,61.717999,62.708000
...,...,...,...,...,...,...
2021-12-02 23:56:00+00:00,61.088001,65.587997,68.468002,67.244003,66.542000,67.928001
2021-12-02 23:57:00+00:00,61.268002,65.426003,68.360001,66.991997,66.632004,68.071999
2021-12-02 23:58:00+00:00,61.268002,65.281998,68.342003,66.919998,66.776001,68.216003


In [111]:
list_line = ["solid", "dot", "dash", "longdash", "dashdot", "longdashdot"]

n_colors = len(list_line)
list_colors = px.colors.sample_colorscale(colorscale, [n/(n_colors -1) for n in range(n_colors)])
random.shuffle(copy.deepcopy(list_colors))

fig = go.Figure()

i_label = 0
for col in data_feat_stats.columns:

    fig.add_trace(go.Scatter(
        mode="lines",
        x=data_feat_stats.index.values,
        y=data_feat_stats[col].values,
        name=col,
        line=dict(
            color=list_colors[i_label],
            dash=list_line[i_label],
        ),
    ))

    i_label+=1

fig.update_layout(
    width=800,
    height=250,
    margin=dict(
        l=0,
        r=0,
        t=30,
        b=0,
    ),
    plot_bgcolor="rgb(245,245,245)",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="center",
        x=0.5,
    )
)

filename = "example_feat_timelag"
pio.write_image(fig, path + "/" + filename + ".svg")
# https://stackoverflow.com/questions/51450134/how-to-convert-svg-to-png-or-jpeg-in-python
image = pyvips.Image.thumbnail(path + "/" + filename + ".svg", 3000)
image.write_to_file(path + "/" + filename + ".png")

fig.show()